In [14]:
import pandas as pd
import numpy as np

In [27]:
lawyers_cases = pd.DataFrame.from_csv("lawyers_cases_1946-2014.csv")

In [16]:
lawyers_cases['side'] = "NaN"
lawyers_cases.loc[(lawyers_cases["petitioner_x"] == 1) & (lawyers_cases["petitioner_y"] == 1), "side"] = 'p'
lawyers_cases.loc[(lawyers_cases["petitioner_x"] == 1) & (lawyers_cases["petitioner_y"].isnull()), "side"] = 'p'
lawyers_cases.loc[(lawyers_cases["petitioner_x"].isnull()) & (lawyers_cases["petitioner_y"] == 1), "side"] = 'p'
lawyers_cases.loc[(lawyers_cases["petitioner_x"] == 1) & (lawyers_cases["other_y"] == 1), "side"] = 'p'
lawyers_cases.loc[(lawyers_cases["other_x"] == 1) & (lawyers_cases["petitioner_y"] == 1), "side"] = 'p'
lawyers_cases.loc[(lawyers_cases["amicus_curiae"] == 1) & (lawyers_cases["petitioner_y"] == 1), "side"] = 'p'

lawyers_cases.loc[(lawyers_cases["respondent_x"] == 1) & (lawyers_cases["respondent_y"] == 1), "side"] = 'r'
lawyers_cases.loc[(lawyers_cases["respondent_x"] == 1) & (lawyers_cases["respondent_y"].isnull()), "side"] = 'r'
lawyers_cases.loc[(lawyers_cases["respondent_x"].isnull()) & (lawyers_cases["respondent_y"] == 1), "side"] = 'r'
lawyers_cases.loc[(lawyers_cases["respondent_x"] == 1) & (lawyers_cases["other_y"] == 1), "side"] = 'r'
lawyers_cases.loc[(lawyers_cases["other_x"] == 1) & (lawyers_cases["respondent_y"] == 1), "side"] = 'r'
lawyers_cases.loc[(lawyers_cases["amicus_curiae"] == 1) & (lawyers_cases["respondent_y"] == 1), "side"] = 'r'

In [17]:
lawyers_cases = lawyers_cases[~((lawyers_cases['side'] == "NaN") & ~((lawyers_cases['other_x'] == 1) | (lawyers_cases['amicus_curiae'] == 1)))]

In [18]:
lawyers_traits = pd.DataFrame.from_csv("lawyer_trait_for_join_1000.csv")

In [19]:
joined = lawyers_cases.merge(lawyers_traits,how='left',
         left_on=['name_merged'],
         right_on=['name'])

In [20]:
traits = list(joined.columns.values)[-40:]

In [21]:
df2 = joined.groupby(['term_merged', 'docket_merged','side'],as_index=False)[traits].mean()

In [22]:
# drop out missing data
df2 = df2[~df2['atypical'].isnull()]

In [23]:
# keep only certain columns
df2 = df2[["term_merged", "docket_merged", "side", "happy", "friendly", "caring", "unhappy", "sociable", "cold", "kind", "unemotional"]]

In [24]:
df2[df2["side"] == "p"].to_csv("petitioner_predictions.csv")

In [25]:
df2[df2["side"] == "r"].to_csv("respondent_predictions.csv")